# Анализ тональности отзывов на фильмы: строим простые модели

*В этом задании вам предлагается начать разбираться с задачей анализа тональности отзывов на примере сентимент-анализа отзывов на фильмы.*

## Загрузка библиотек

In [22]:
import nltk
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
import operator

## Загрузка данных

In [7]:
nltk.download('movie_reviews')

[nltk_data] Downloading package movie_reviews to
[nltk_data]     /home/sergio/nltk_data...
[nltk_data]   Unzipping corpora/movie_reviews.zip.


True

In [8]:
negids = movie_reviews.fileids('neg')
posids = movie_reviews.fileids('pos')

## Задание №1
*Создайте список из текстов всех имеющихся отзывов, а также список с классами, которые будет использовать ваш классификатор - 0 для негативных отзывов и 1 для позитивных.*

In [9]:
negfeats = [" ".join(movie_reviews.words(fileids=[f])) for f in negids]
posfeats = [" ".join(movie_reviews.words(fileids=[f])) for f in posids]

texts = negfeats + posfeats
labels = [0] * len(negfeats) + [1] * len(posfeats)

## Задание №2
*Подсчитайте количество отзывов в выборке.*

In [14]:
ans1 = str(len(texts))
print(f"Количество отзывов в выборке: {ans1}")

Количество отзывов в выборке: 2000


In [13]:
with open('ans1.txt', 'w') as f:
    f.write(ans1)

## Задание №3
*Подсчитайте долю класса 1 в выборке.*

In [16]:
ans2 = str(len(posfeats) / len(texts))
print(f"Доля положительных отзывов: {ans2}")

Доля положительных отзывов: 0.5


In [17]:
with open('ans2.txt', 'w') as f:
    f.write(ans2)

## Задание №4
*Импортируйте CountVectorizer из sklearn.feature_extraction.text. Попробуйте использовать его с настройками по умолчанию для того, чтобы получить признаковое представление каждого текста. Скорее всего, попытка не увенчается успехом. Разберитесь, в чем причина, и добейтесь того, чтобы метод fit_transform у CountVectorizer успешно отрабатывал. Подсчитайте количество признаков в CountVectorizer. Никакой предварительной обработки текста (удаление стоп-слов, нормализация слов) на этом шаге делать не надо, в качестве признаков должны использоваться частоты слов.*

In [18]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(texts)

In [19]:
ans3 = str(X.shape[1])
print(f"Количество признаков в CountVectorizer: {ans3}")

Количество признаков в CountVectorizer: 39659


In [20]:
with open('ans3.txt', 'w') as f:
    f.write(ans3)

## Задание №5
*Соберите pipeline из CountVectorizer и LogisticRegression c настройками по-умолчанию и с помощью cross_val_score (также со стандартными настройками) оцените получаемое "из коробки" качество по accuracy.*

In [23]:
pipeline = Pipeline([
    ('count_vect', CountVectorizer()), 
    ('log_reg', LogisticRegression(solver='liblinear')
    )])

In [27]:
ans4 = str(cross_val_score(pipeline, texts, labels, scoring='accuracy', cv=3).mean())
print(f"Accuracy на кросс-валидации - {ans4}")

Accuracy на кросс-валидации - 0.8360216503929078


In [28]:
with open('ans4.txt', 'w') as f:
    f.write(ans4)

## Задание №6
*Аналогично accuracy, оцените качество по ROC AUC.*

In [29]:
ans5 = str(cross_val_score(pipeline, texts, labels, scoring='roc_auc', cv=3).mean())
print(f"ROC AUC на кросс-валидации - {ans5}")

ROC AUC на кросс-валидации - 0.9107764937833774


In [30]:
with open('ans5.txt', 'w') as f:
    f.write(ans5)

## Задание №7
*Обучите логистическую регрессию на всей доступной вам выборке и выведите 5 наиболее важных для модели признаков (подумайте, какие именно признаки стоит считать такими). Вам могут пригодиться метод get_feature_names() или поле vocabulary_ у класса CountVectorizer.*

In [34]:
log_reg = LogisticRegression(solver='liblinear')
log_reg.fit(X, labels)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='liblinear',
          tol=0.0001, verbose=0, warm_start=False)

In [42]:
coeffs = log_reg.coef_[0]
ans6 = [vectorizer.get_feature_names()[list(coeffs).index(i)] for i in sorted(coeffs)[:5]]
ans6

['bad', 'unfortunately', 'worst', 'waste', 'nothing']

In [43]:
with open('ans6.txt', 'w') as f:
    f.write(' '.join(ans6[:2]))